# ELEC 576 / COMP 576 - Fall 2018 Assignment 2

## 1 Visualizing a CNN with CIFAR10

### a) CIFAR10 Dataset

Load CIFAR10 Dataset:

In [1]:
from trainCifarStarterCode import *
import numpy as np
import imageio

ntrain = 1000 # per class
ntest = 100 # per class
nclass = 10 # number of classes
imsize = 28
nchannels = 1

Train = np.zeros((ntrain*nclass, imsize, imsize, nchannels))
Test = np.zeros((ntest*nclass, imsize, imsize, nchannels))
LTrain = np.zeros((ntrain*nclass, nclass))
LTest = np.zeros((ntest*nclass, nclass))

itrain = -1
itest = -1
for iclass in range(0, nclass):
    for isample in range(0, ntrain):
        path = './CIFAR10/Train/%d/Image%05d.png' % (iclass,isample)
        im = imageio.imread(path); # 28 by 28
        im = im.astype(float)/255
        itrain += 1
        Train[itrain,:,:,0] = im
        LTrain[itrain,iclass] = 1 # 1-hot lable
    for isample in range(0, ntest):
        path = './CIFAR10/Test/%d/Image%05d.png' % (iclass,isample)
        im = imageio.imread(path); # 28 by 28
        im = im.astype(float)/255
        itest += 1
        Test[itest,:,:,0] = im
        LTest[itest,iclass] = 1 # 1-hot lable

print(Train.shape, LTrain.shape, Test.shape, LTest.shape)

(10000, 28, 28, 1) (10000, 10) (1000, 28, 28, 1) (1000, 10)


### b) Train LeNet5 on CIFAR10

Model architecture:

    • Convolutional layer with kernel 5 x 5 and 32 filter maps followed by ReLU
    • Max Pooling layer subsampling 2
    • Convolutional layer with kernel 5 x 5 and 64 filter maps followed by ReLU
    • Max Pooling layer subsampling by 2
    • Fully Connected layer that has input 7*7*64 and output 1024
    • Fully Connected layer that has input 1024 and output 10 (for the classes)
    • Softmax layer (Softmax Regression + Softmax Nonlinearity)

In [4]:
class LeNet5(object):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
        # --------------------------------------------------
        # model
        #create your model

        # first convolutional layer
        h_pool1 = conv_layer(self.X, [5, 5, 1, 32], 'conv1', tf.nn.relu)

        # second convolutional layer
        h_pool2 = conv_layer(h_pool1, [5, 5, 32, 64], 'conv2', tf.nn.relu)

        # densely connected layer
        h_fc1 = fc_layer(h_pool2,[7 * 7 * 64, 1024], 'fc1', tf.nn.relu)

        # dropout
        self.keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = dropout(h_fc1, self.keep_prob, 'drop')

        # softmax
        self.y_ = fc_layer(h_fc1_drop, [1024, 10], 'fc2', None)#tf.nn.softmax)

        
    def loss(self, learning_rate):
        # --------------------------------------------------
        # loss
        #set up the loss, optimization, evaluation, and accuracy

#         self.cross_entropy = tf.reduce_mean(-tf.reduce_sum(self.y * tf.log(self.y_), reduction_indices=[1]), name="loss_function")
        self.cross_entropy = tf.losses.softmax_cross_entropy(self.y, self.y_) + \
                             tf.losses.get_regularization_loss()
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cross_entropy)

        correct_prediction = tf.cast(tf.equal(tf.argmax(self.y_,1), tf.argmax(self.y,1)), tf.float32, name="correct_prediction")
        self.accuracy = tf.reduce_mean(correct_prediction, name="accuracy")

        
    def train(self, X, y, batchsize, nepoch):
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            nsamples = y.shape[0]
            perm = np.arange(nsamples)
            steps = np.floor(nsamples/batchsize).astype(int)
            for i in range(nepoch): # try a small iteration size once it works then continue
                np.random.shuffle(perm)
                for n in range(steps):
                    batch_ind = perm[n*batchsize:(n+1)*batchsize]
                    self.optimizer.run(feed_dict={
                        self.X: X[batch_ind], 
                        self.y: y[batch_ind], 
                        self.keep_prob: 0.5}) # dropout only during training

                if i%10 == 0 or i==nepoch-1:
                    #calculate train accuracy and print it
                    train_acc, train_loss = sess.run([self.accuracy, self.cross_entropy], 
                                                     feed_dict={self.X: X[batch_ind], 
                                                                self.y: y[batch_ind], 
                                                                self.keep_prob: 1.0})
                    test_acc, test_loss = sess.run([self.accuracy, self.cross_entropy], 
                                                   feed_dict={self.X: Test, self.y: LTest, self.keep_prob: 1.0})

                    print("epoch %d, train accuracy %g, train loss %g, test loss %g"%(i, train_acc, train_loss, test_loss))

            # --------------------------------------------------
            # test

            print("test accuracy %g"%test_acc)


In [5]:
batchSize = 300
numberOfEpoch = 100
learningRate = 1e-4

#tf variable for the data, remember shape is [None, width, height, numberOfChannels] 
tf_data = tf.placeholder(tf.float32, [None, imsize, imsize, nchannels]) 

#tf variable for labels
tf_labels = tf.placeholder(tf.float32, [None, nclass]) 

# --------------------------------------------------
# optimization

model = LeNet5(tf_data, tf_labels)
model.loss(learning_rate = learningRate)
model.train(Train, LTrain, batchSize, numberOfEpoch)


epoch 0, train accuracy 0.2, train loss 2.19732, test loss 2.2492
epoch 10, train accuracy 0.426667, train loss 1.69949, test loss 1.73424
epoch 20, train accuracy 0.563333, train loss 1.37356, test loss 1.59686
epoch 30, train accuracy 0.606667, train loss 1.2606, test loss 1.51648
epoch 40, train accuracy 0.67, train loss 1.06117, test loss 1.4468
epoch 50, train accuracy 0.7, train loss 0.986593, test loss 1.40889
epoch 60, train accuracy 0.723333, train loss 0.907592, test loss 1.37879
epoch 70, train accuracy 0.843333, train loss 0.630851, test loss 1.36931
epoch 80, train accuracy 0.85, train loss 0.656864, test loss 1.37183
epoch 90, train accuracy 0.863333, train loss 0.512718, test loss 1.35748
epoch 99, train accuracy 0.883333, train loss 0.466778, test loss 1.36261
test accuracy 0.525
